In [125]:
import pandas as pd
import numpy as np
from matplotlib import style

from sklearn import preprocessing

from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.cluster import Birch
from sklearn.neural_network import BernoulliRBM
from sklearn.mixture import BayesianGaussianMixture
from sklearn.manifold import Isomap

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm

from sklearn.ensemble import AdaBoostClassifier
import random

In [126]:
def adaboost_clf(Y_train, X_train, M, err_pos, err_neg):
    clf = DecisionTreeClassifier(max_depth=1, random_state=1)
    n_train = len(X_train)
    # Initialize weights
    w = np.ones(n_train) / n_train
    clf_list = []

    for i in range(M):
        # Fit a classifier with the specific weights
        clf.fit(X_train, Y_train, sample_weight=w)
        pred_train_i = clf.predict(X_train)

        miss = []
        # Indicator function
        for x in range(len(pred_train_i)):
            if pred_train_i[x]==Y_train[x]:
                miss.append(0)
            else:
                miss.append(1) 

        # Error
        err_m = np.dot(w, miss) / sum(w)
        
        # Divisor
        dev = 1 - err_pos - err_neg

        # Alpha
        alpha_m = 0.5 * np.log((1 - err_m) / float(err_m))

        #noisy correction
        miss2 = []
        for x in range(len(miss)):
            if miss[x] == 0:
               if Y_train[x] > 0:
                  miss2.append(1 - err_neg + err_pos)
               else:
                   miss2.append(1 + err_neg - err_pos)
            else:
                if Y_train[x] > 0:
                    miss2.append(-(1 - err_neg + err_pos))
                else:
                    miss2.append(-(1 + err_neg - err_pos))

        w = np.multiply(w, np.exp([(float(x)/dev) * alpha_m for x in miss2]))
        clf_list.append((clf, alpha_m))
    return clf_list

def classify_adaBoosting(clf_list, X):
    total = 0
    for clf in clf_list:
        if clf[0].predict(X)>0:
            total += clf[1]
        else:
            total -= clf[1]
    if total > 0:
        return 1
    else:
        return 0

In [127]:
#read in the data
df = pd.read_csv('data.csv')

#columns to drop
df = df.drop(['id'], axis=1)

df.sample(frac=1)
#gets rid of ? and one hot encoding for all columns that need it
index = []
count = 0
for val in range(len(df.ix[:,0])):
    flag = False
    for column in df:
        if df[column][val] == '?':
            flag = True
            break
    if flag:
        continue
    if count<1000:
        index.append(val)
        count += 1
df = df[df.index.isin(index)]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


In [128]:
#gets all columns which are not ints and integer encodes them
obj_df = df.select_dtypes(include=['object']).copy()
for column in obj_df:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    df[column] = le.transform(df[column])

In [129]:
#normalize all points between [0,1]
x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [130]:
train, test = np.split(df.sample(frac=1), [int(.6*len(df))])
unlabel, label = np.split(train.sample(frac=1), [int(.8*len(train))])
test = test.values.tolist()
nolabels1 = unlabel.values.tolist()
del unlabel[0]
nolabels = unlabel.values.tolist()
labels = label.values.tolist()

In [131]:
true = {}
for row in nolabels1:
    true[tuple(row[1:])] = row[:1][0]

In [132]:
#train f1...fn classifiers on labelled data, will use 6 types: decision stumps, knn, svm, guassian mixture mode
#                                                               native bayes, logistic regression
x = []
y = []
x_true = []
y_true = []
for row in labels:
    x.append(row[1:])
    y.append(row[:1][0])
    x_true.append(row[1:])
    y_true.append(row[:1][0])
clf1 = svm.SVC(kernel='linear', gamma='scale').fit(x, y)
clf2 = KNeighborsClassifier(n_neighbors=3).fit(x, y)
clf3 = DecisionTreeClassifier(splitter= 'random').fit(x, y)
clf4 = DecisionTreeClassifier(splitter= 'random').fit(x, y)
clf5 = DecisionTreeClassifier(splitter= 'random').fit(x, y)
clf6 = GaussianMixture(n_components = 2, init_params= 'random').fit(x,y)
clf7 = GaussianMixture(n_components = 2, init_params= 'random').fit(x,y)
clf8 = GaussianMixture(n_components = 2, init_params= 'random').fit(x,y)
clf9 = GaussianNB().fit(x,y)
clf10 = LogisticRegression(solver='liblinear').fit(x,y)
classifiers = [clf1, clf2, clf3, clf4, clf5, clf6, clf7, clf8, clf9, clf10]

In [133]:
# make csv in form of rowNumber, clfNumber, clf prediction on that row
answers = []
for point in range(len(nolabels)):
    for clf in range(len(classifiers)):
        answers.append([point, clf, classifiers[clf].predict([nolabels[point]])])
        
count = 0
f = open("answer_file.csv", "w")
f.write('question,worker,answer;\n')
for answer in answers:
    count += 1
    f.write(str(answer[0]) + ',' + str(answer[1]) + ',' + str(int(answer[2]))+'\n')
f.close()
p = open("result_file.csv", "w")   
p.close()

In [134]:
#run VI BP
import subprocess
subprocess.call(["python", "run.py", "methods/c_EM/method.py", "answer_file.csv", "result_file.csv","decision-making"])

0

In [135]:
#extract results, get noisy labels and 
filepath = "result_file.csv"
noisy_labels = []
with open(filepath) as fp:  
    for line in fp:
        questionAnswer = line.split(',')
        noisy_labels.append(questionAnswer)

In [136]:
#assign noisy label to proper row
#combine noisy lables to real labels and randomize
df_noise_x = [] 
df_noise_y = []
for question in noisy_labels:
    if question[0].rstrip() == 'question':
        continue
    df_noise_x += [nolabels[int(question[0].rstrip())]]
    df_noise_y.append(int(question[1].rstrip()))


count_vi_pos = 0
count_vi_neg = 0
for el in range(len(df_noise_x)):
    if true[tuple(df_noise_x[el])] != df_noise_y[el]:
        if df_noise_y[el] > 0:
            count_vi_pos += 1
        else:
            count_vi_neg += 1

#Postive and Negative Error (Hard Calculation)
err_pos = count_vi_pos/len(df_noise_x)
err_neg = count_vi_neg/len(df_noise_x)
print(count_vi_pos, count_vi_neg)

2 13


In [137]:
df_noise_x += x
df_noise_y += y

df_noise = []
for el in range(len(df_noise_x)):
    new = df_noise_x[el]
    new.append(df_noise_y[el])
    df_noise.append(new)

#need to shuffle the data
random.shuffle(df_noise)

df_noise_x = []
df_noise_y = []
for row in df_noise:
    df_noise_x.append(row[:-1])
    df_noise_y.append(row[-1:][0])

In [138]:
#Run Adaboosting Algo with noise correction
clf_list_noisy = adaboost_clf(df_noise_y, df_noise_x, 20, err_pos, err_neg)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: overflow encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/tree/tree.py:276: RuntimeWarning: invalid value encountered in double_scalars
  np.sum(sample_weight))


In [139]:
clf_list_clean_100 = adaboost_clf(y_true, x_true, 20, err_pos, err_neg)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/tree/tree.py:276: RuntimeWarning: invalid value encountered in double_scalars
  np.sum(sample_weight))


In [140]:
#Ada boosting on noisy data error rate
errors = []
count1 = 0
for point in test:
    est = classify_adaBoosting(clf_list_noisy,[point[:-1]])
    true = int(point[-1:][0])
    if est == true:
        errors.append([point[:-1],0])
    else:
        count1 += 1
        errors.append([point[:-1],1])

#Ada boosting on 500 supervised data error rate
errors = []
count = 0
for point in test:
    #est = bdt1.predict([point[:-1]])
    est = classify_adaBoosting(clf_list_clean_100, [point[:-1]])
    true = int(point[-1:][0])
    #est = int(est[0])
    if est == true:
        errors.append([point[:-1],0])
    else:
        count += 1
        errors.append([point[:-1],1])
print(count1/len(test),count/len(test))

0.0 0.0
